In [40]:
%%bash --out windowID --err err
# find simulator window id
xdotool search --name 'self_driving_car_nanodegree_program'

In [41]:
if (windowID == ''):
    print("ERROR: no window id found")
else:
    print("found following windowID: " + str(windowID))
    windowID = str(windowID)

found following windowID: 41943046



In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import subprocess
import time
import os
%matplotlib inline  
sleepTime = 0.5


def alarm():
    duration = 0.5  # second
    freq = 240  # Hz
    os.system('play --no-show-progress --null --channels 1 synth %s sine %f' % (duration, freq))

def startSimulation():
    # start current simulation
    # uncomment to activate window
    #subprocess.call(["xdotool", "windowactivate", windowID])
    #time.sleep(sleepTime)
    subprocess.call(["xdotool", "mousemove", "596", "359"])
    time.sleep(sleepTime)
    subprocess.call(["xdotool", "click", "1"])
    time.sleep(sleepTime)
    subprocess.call(["xdotool", "click", "1"])
    time.sleep(sleepTime)
    subprocess.call(["xdotool", "click", "1"])
    time.sleep(sleepTime)
    subprocess.call(["xdotool", "mousemove", "369", "445"])
    time.sleep(sleepTime)
    subprocess.call(["xdotool", "click", "1"])
    time.sleep(sleepTime)
    

def exitSimulation():
    # exit current simulation
    # uncomment to activate window
    #subprocess.call(["xdotool", "xdotool", "windowactivate", windowID])
    #time.sleep(sleepTime)
    subprocess.call(["xdotool", "mousemove", "389", "41"])
    time.sleep(sleepTime)
    subprocess.call(["xdotool", "click", "1"]) 
    time.sleep(sleepTime)
    subprocess.call(["xdotool", "key", "Escape"])
    time.sleep(sleepTime)

# run the cpp PID controller
def startPIDcontroller(params, N):
    process = subprocess.Popen(["./build/pid", str(params[0]), str(params[1]), str(params[2]), str(N)], stdout=subprocess.PIPE)
    out, err = process.communicate()
    return out

# NOTE: We use params instead of tau_p, tau_d, tau_i
def run(params, n=1200):
    startSimulation()
    err = startPIDcontroller(params, n)
    print(err)
    writeRunToFile("./twiddle.out", params, float(err)/n)
    exitSimulation()
    return float(err)/n

# write each simulator run to a file
def writeRunToFile(fn, p, best_err):
    f = open(fn, "a")
    f.write("new run: " + str(p) + ": " + str(best_err) + "\n")
    f.close()
    
# write parameter to a file
def writeParamsToFile(fn, dp, weights, p, best_err):
    f = open(fn, "a")
    f.write("-->changed dp: "+ str(dp) + " (sum: " + str(sum([float(a)*b for a,b in zip(dp,weights)])) + ")" + 
            ", p: " + str(p) + ", best_err: " + str(best_err) + "\n")
    f.close()

# Make this tolerance bigger if you are timing out!
def twiddle(start_p = [0.1, 0.001, 1.0], 
            start_dp = [1,1,1],
            weights=[10,1000,1], 
            tol=0.01): 
    p = start_p
    dp = start_dp
    dp = [float(a)/b for a,b in zip(dp,weights)]
    best_err = run(p)
    writeParamsToFile("./twiddle.out", dp, weights, p, best_err)
    
    # twiddle loop
    while sum([float(a)*b for a,b in zip(dp,weights)]) > tol:
        for i in range(len(dp)):
            p[i] += dp[i]
            err = run(p)
            if err < best_err:
                best_err = err
                dp[i] *= 1.1
                writeParamsToFile("./twiddle.out", dp, weights, p, best_err)
                
            else:
                p[i] -= 2*dp[i]
                err = run(p)
                if err < best_err:
                    best_err = err
                    dp[i] *= 1.1
                    writeParamsToFile("./twiddle.out", dp, weights, p, best_err)
                else:
                    p[i] += dp[i];
                    dp[i]*=0.9
                    writeParamsToFile("./twiddle.out", dp, weights, p, best_err)
    
    return p, best_err


params, err = twiddle()
print("Final twiddle error = {}".format(err))


b'680.648'
b'330.793'
b'329.676'


In [29]:
params

[0.18036159739366794, 9.999999999998899e-05, 4.286364811121022]